# Fernando Martinelli Ramacciotti 301002
### MPE | Econometria das Séries Financeiras - Lista de Exercício 1
fernandoramacciotti@gmail.com

In [1]:
import numpy as np
import pandas as pd
import os
from scipy import stats
import plotly
plotly.offline.init_notebook_mode()


import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot

import plotly.offline as po
import plotly.plotly as py
import plotly.graph_objs as go

%matplotlib inline

funds = pd.ExcelFile('./dados/funds.xlsx')
risk_free = pd.ExcelFile('./dados/risk_free.xlsx')
factors = pd.ExcelFile('./dados/factors.xlsx')
ibov = pd.ExcelFile('./dados/IBOVESPA.xlsx') 

C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [2]:
# consolidar todas os fundos e fatores em um unico DataFrame

data = pd.DataFrame()

for asset_class in funds.sheet_names:
    df_1 = funds.parse(asset_class, index_col = 0, skiprows = 1)
    df_1.columns = ['fund_' + asset_class]
    df_1.index.name = 'Dates'
    data = pd.concat([data, df_1],join='outer',axis=1)

for asset_class in risk_free.sheet_names:
    df_2 = risk_free.parse(asset_class, index_col = 0,skiprows = 1)
    df_2.columns = [asset_class]
    df_2.index.name = 'Dates'
    data = pd.concat([data, df_2], join = 'outer', axis = 1)  

for asset_class in factors.sheet_names:
    df_3 = factors.parse(asset_class, index_col = 0,skiprows = 1)
    df_3.columns = [asset_class]
    df_3.index.name = 'Dates'
    data = pd.concat([data, df_3], join = 'outer', axis = 1)
    
for asset_class in ibov.sheet_names:
    df_4 = ibov.parse(asset_class, index_col = 0,skiprows = 1)
    df_4.columns = ['bench_' + asset_class]
    df_4.index.name = 'Dates'
    data = pd.concat([data, df_4], join = 'outer', axis = 1)

In [3]:
# calculando vol mensal
vols = pd.DataFrame()
for col in data.columns:
    if 'Factor' in col:
        # factors j[a estao como retornos, portanto multiplicamos a janela de 21 por raiz(21)]
        vols[col + '_vol'] = np.log(1 + data[col]-data['risk_free']).rolling(window = 21).std() * np.sqrt(21)
    else:
        # vols do excesso de retorno mensal, i.e. * raiz(21)
        vols[col + '_vol'] = np.log((data[col]-data['risk_free']) / (data[col].shift(1)-data['risk_free'].shift(1))).rolling(window = 21).std() * np.sqrt(21)

In [4]:
# resampling para monthly basis
data = data.resample('BM').last()
vols = vols.resample('BM').last()


## Grafico Index

In [5]:
# plot dos indices dos fundos
plot_data = [{
            'x': data.index,
            'y': data[col],
            'name': col}  for col in data.columns if 'fund' in col]

layout = go.Layout(title = 'Indices')

fig = go.Figure(data = plot_data, layout = layout)
po.iplot(fig)


## Grafico do Excesso de Retornos

In [15]:
# retornos dos fundos e benchmarks (fatores assumidos ja como retornos)
Returns = data.loc[:, ~data.columns.str.contains('Factor')].pct_change(1)

## calculando os pesos de cada fator no risk-parity benchmark
peso fator = (1/(volatilidade do fator no periodo)) / (soma dos (1/vol) de todos os fatores))

In [16]:
# calculando os pesos de cada fator no risk-parity benchmark
# peso fator = (1/(volatilidade do fator no periodo)) / (soma dos (1/vol) de todos os fatores))

factor_wgt = pd.DataFrame()
for col in vols.columns:
    if 'Factor' in col:
        factor_wgt[col.replace('_vol', '') + '_1overvol'] = 1 / vols[col]
        
factor_wgt['sum_1overvol'] = factor_wgt.sum(axis = 1)
for col in factor_wgt.columns:
    if 'Factor' in col:
        factor_wgt[col.replace('_1overvol', '')] = factor_wgt[col] / factor_wgt['sum_1overvol']

for col in factor_wgt.columns:
    if '1over' in col:
        factor_wgt.drop(col, axis = 1, inplace = True)

## Criando os Benchmarks

### i) IBOV (ja adicionado na serie de Returns)

### ii) 70% IBOV + 30% Risk Free

In [17]:
Returns['bench_70ibov_30riskfree'] = 0.7 * Returns['bench_IBOV'] + 0.3 * Returns['risk_free']

### iii) risk-parity dos 5 fatores com os pesos calculados acima

In [18]:
aux_factors = pd.DataFrame()
for col in factor_wgt:
        aux_factors[col] = data[col] * factor_wgt[col]


Returns['bench_riskp_factors'] = aux_factors.sum(axis = 1)

Grafico Excesso de retorno

In [41]:
EReturns = pd.DataFrame()
for asset_class in Returns.columns:
            EReturns[asset_class] = Returns[asset_class] - Returns['risk_free']

EReturns = EReturns.drop('risk_free', axis = 1)

In [42]:
plot_data_funds = [{
        'x': EReturns.index,
        'y': EReturns[col],
        'name': col, 
        'mode' : 'markers',
        'line': {'width': 1}
          }  for col in EReturns.columns if 'fund' in col]

plot_data_benchs = [{
        'x': EReturns.index,
        'y': EReturns[col],
        'name': col,
        'line': {'width': 3}
          }  for col in EReturns.columns if 'bench' in col]


layout = go.Layout(title = 'Excesso de retorno fundos vs. Benchmarks')

plot_data = plot_data_funds + plot_data_benchs

fig = go.Figure(data = plot_data, layout = layout)
po.iplot(fig)

Nota-se que pontualmente os benchmarks são batidos em alguns meses.
Para uma melhor comparação, vamos plotar os retornos acumulados.

## Grafico Retornos acumulados de um início em comum (2014.05)

In [43]:
# Retornos acumulados (sem inicio em comum)
CumReturns = pd.DataFrame()
for i in EReturns.columns:
    CumReturns[i] = (1 + EReturns[i]).cumprod() - 1
    
#po.iplot([{
#        'x': CumReturns.index,
#        'y': CumReturns[col],
#        'name': col
#          }  for col in CumReturns.columns if 'fund' in col], filename='CumXsRet')

In [44]:
plot_data_funds = [{
        'x': CumReturns.index,
        'y': CumReturns[col],
        'name': col,
        'line': {'width': 1}
          }  for col in CumReturns.columns if 'fund' in col]

plot_data_benchs = [{
        'x': CumReturns.index,
        'y': CumReturns[col],
        'name': col,
        'line': {'width': 3}
          }  for col in CumReturns.columns if 'bench' in col]


layout = go.Layout(title = 'Excesso de retorno acumulado fundos vs. Benchmarks')

plot_data = plot_data_funds + plot_data_benchs

fig = go.Figure(data = plot_data, layout = layout)
po.iplot(fig)

## Grafico Retornos acumulados do inicio em comum (2014.05)

In [45]:
CommonEReturns = EReturns.dropna()
CumReturns = pd.DataFrame()
for col in CommonEReturns.columns:
    CumReturns[col] = (1 + CommonEReturns[col]).cumprod() - 1
    

In [46]:
plot_data_funds = [{
        'x': CumReturns.index,
        'y': CumReturns[col],
        'name': col,
        'mode': 'markers',
        'line': {'width': 1}
          }  for col in CumReturns.columns if 'fund' in col]

plot_data_benchs = [{
        'x': CumReturns.index,
        'y': CumReturns[col],
        'name': col,
        'line': {'width': 3}
          }  for col in CumReturns.columns if 'bench' in col]


layout = go.Layout(title = 'Excesso de retorno acumulado fundos vs. Benchmarks')

plot_data = plot_data_funds + plot_data_benchs

fig = go.Figure(data = plot_data, layout = layout)
po.iplot(fig)

Calculo do sharpe ratio de cada fundo

In [144]:
sharpe = pd.DataFrame()
for col in CommonEReturns.columns:
    #if 'fund' in col:
        sharpe['sharpe_' + col] = ((1 + CommonEReturns[col].rolling(window = CommonEReturns.shape[0]).mean()) ** (12 / CommonEReturns.shape[0] + 1) - 1) / (CommonEReturns[col].rolling(window = CommonEReturns.shape[0]).std() * np.sqrt(21 / (CommonEReturns.shape[0] + 1)))
        #CommonEReturns[col].mean() # / (CommonEReturns[col].std() * np.sqrt(CommonEReturns.shape[0]))
sharpe = sharpe.dropna()
sharpe

sharpe_fund_atmos  sharpe_fund_msquare  sharpe_fund_jgp  \
Dates                                                                 
2017-03-31           0.721684             0.316321          0.27161   

            sharpe_fund_dynamo  sharpe_fund_nucleo  sharpe_fund_spx  \
Dates                                                                 
2017-03-31            0.472868             0.43849         0.551825   

            sharpe_fund_ip  sharpe_fund_tempo  sharpe_fund_oceana  \
Dates                                                               
2017-03-31         0.67882           0.288988            0.251546   

            sharpe_fund_bogari  sharpe_fund_const  sharpe_fund_gavea  \
Dates                                                                  
2017-03-31            0.354579           0.172985           0.148041   

            sharpe_fund_squadra  sharpe_fund_tarpon  sharpe_bench_IBOV  \
Dates                                                                    
2017-03-31             0.258759           -0.320022           0.150905   

            sharpe_bench_70ibov_30riskfree  sharpe_bench_riskp_factors  
Dates                                                                   
2017-03-31                        0.150856                   -0.198897

### Fundos que bateram o benchmark
#### i) IBOV

In [146]:
import plotly.figure_factory as ff

table = pd.DataFrame()

table['Annualized CumReturn'] = round(((1 + CumReturns.iloc[-1, :]) ** (12 / (CumReturns.shape[0] + 1)) - 1 )* 100, 2)
table['Sharpe Ratio'] = sharpe.iloc[-1, :].values.round(2)
table['Beat IBOV'] = np.where(CumReturns.iloc[-1, :] > CumReturns['bench_IBOV'].iloc[-1], 'yes', 'no')
table['Beat 70IBOV_30Rf'] = np.where(CumReturns.iloc[-1, :] > CumReturns['bench_70ibov_30riskfree'].iloc[-1], 'yes', 'no')
table['Beat RiskParity Factors'] = np.where(CumReturns.iloc[-1, :] > CumReturns['bench_riskp_factors'].iloc[-1], 'yes', 'no')
table.index.name = 'Fund'

In [147]:
import plotly.figure_factory as ff

table_2 = ff.create_table(table, index = True)

for i in range(len(table_2.layout.annotations)):
    try:
        if table_2.layout.annotations[i]['text'] == 'yes':
            table_2.layout.annotations[i].update(font=dict(color='green'))
        elif table_2.layout.annotations[i]['text'] == 'no':
            table_2.layout.annotations[i].update(font=dict(color='red'))
    except: #if cell has no value
        pass

po.iplot(table_2)

No período analisado, 2014.05 - 2017.04, o único fundo que não bateu nenhum benchmark foi a Tarpon e o Gávea só bateu o risk_parity.
O melhor fundo, em termos de excesso de retorno no período e Sharpe Ratio, foi o Atmos, seguido pelo IP e Dynamo.